# Using AI21 Contextual Answers Support Emergency Call Dispatchers

When receiving calls, emergency call dispatchers must quickly extract key information from civillians who are reporting an emergency. Given the stress and complex situation, callers are frequently confused or may not understand what is going on around them.

In the context of emergency calls, dispatchers must find out the address where the caller is calling from, as well as the nature of the emergency. In the context of calls related to fires, dispatchers must quickly decide if the call is about a [working fire](https://en.wikipedia.org/wiki/Multiple-alarm_fire), which would require a coordinated response from multiple fire departments, or a minor incident or false alarm. 

This notebook demonstrates how the [AI21 Contextual Answers](https://www.ai21.com/studio/contextual-answers) Task Specific model can be used to surface answers to these two important questions for dispatchers. Given a sample transcript of a call, Contextual Answers will surface the needed information. Importantly, such answers must fulfill two criteria:

1. If no answer is present, no answer is surfaced
2. The answer must be current to what the caller is saying **now**. This can alert the dispatcher if anything changes while the caller is on the line.

### Load Library

In [1]:
from ai21 import AI21Client

client = AI21Client() #uses the environment variable 
#If you wish to pass your api key as a string, you can use the following code

import json
import os
os.environ["AI21_LOG_LEVEL"] = "DEBUG"

from ai21 import AI21Client
client = AI21Client()
#client = AI21Client(api_key='my_api_key')

Define Functions

In [2]:
def read_file_into_rolling_window(filename, window_size=2):
# Initialize an empty list to store the rolling window lists
    result = []
    
    # Open the file and read lines
    with open(filename, 'r') as file:
        content = file.read().splitlines()  # Read lines and split by newline, creating a list of lines
    
        # Process each line in the rolling window fashion
        for i in range(len(content)):
            if i < window_size:
                # For the initial lines, add them progressively
                result.append("\n".join(content[:i+1]))
            else:
                # For subsequent lines, create a rolling window of the specified size
                result.append("\n".join(content[i-window_size+1:i+1]))
    
    return result
def call_ca(context,question):
    response = client.answer.create(
      context=context,
      question=question)
    answer=response.answer
    if response.answer is None:
        answer="None"
    return(answer)



### Load Transcript
Now we will load the transcript, in a live situation, audio can be passed to transcription services, such as [AWS Transcribe](https://aws.amazon.com/transcribe/) to get real-time transcriptions. In order to emulate a real-time transcription, where we only have access to part of the transcript any any given time, we will split the transcript into sections. Contextual Answers thus only analyzes **the callers current response, and the dispatchers previous comment**

In [3]:
filename = 'sample_emergency_call.txt'  # Specify the path to your file
rolling_transcript= read_file_into_rolling_window(filename)
#print(rolling_transcript[0:5])


Now we will analyze the transcript. Looking to see if a working fire or the callers address is reported. Note that Contextual Answers correctly finds that the earlier in the transcript, the caller reports a fire, but then *changes their mind* and instead notes that it is a false alarm. Contextual Answers also correctly does not surface the address until it actually shows up in the transcript.

In [4]:
for current_transcript in rolling_transcript:
    address_question="What is the caller's address?"
    address_present=call_ca(current_transcript, address_question)

    active_fire_question="Is the caller reporting a fire in the building?"
    active_fire_present=call_ca(current_transcript, active_fire_question)
    
    print("Address: ",address_present)
    print("Fire:    ",active_fire_present)
    print("_____\n")

Address:  None
Fire:     None
_____

Address:  None
Fire:     None
_____

Address:  None
Fire:     None
_____

Address:  None
Fire:     None
_____

Address:  None
Fire:     None
_____

Address:  The caller's address is 1234 Cherry Road.
Fire:     None
_____

Address:  The caller's address is 1234 Cherry Road.
Fire:     None
_____

Address:  None
Fire:     The caller is reporting a fire in the building.
_____

Address:  None
Fire:     The caller is reporting a fire in the building.
_____

Address:  None
Fire:     The caller is not reporting a fire, but a kitchen accident.
_____

Address:  None
Fire:     The caller is not reporting a fire in the building, but a kitchen accident.
_____

Address:  None
Fire:     None
_____



### Conclusion

This example shows how contextual answers can be used for analyzing not only whole transcripts, but also partial or incomplete transcripts, surfacing answers to questions, and even switching anwers depending on the context.